In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [52]:
from textblob import TextBlob

In [7]:
with open('smsspamcollection/SMSSpamCollection','r') as f:
    datafile = f.readlines()

In [10]:
messages = []
for i in datafile:
    messages.append(i.split('\t'))

In [11]:
len(messages)

5574

In [14]:
messages[20]

['ham', 'Is that seriously how you spell his name?\n']

In [17]:
df = pd.DataFrame(messages,columns=['spam','text'])

In [18]:
df.head()

,spam,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...\n
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [20]:
df['spam'] = df.spam.apply(lambda x: 1 if x=='spam' else 0)

In [21]:
df.head()

,spam,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [22]:
df.groupby('spam').count()

,text
spam,
0,4827
1,747


In [28]:
len(df)

5574

In [75]:
X = df['text']
y = df['spam']

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [77]:
X_train.shape,X_test.shape, y_train.shape, y_test.shape

((5295,), (279,), (5295,), (279,))

In [80]:
X_train.shape

(5295,)

In [81]:
X_train[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n'

In [82]:
X_train.index

Int64Index([1047, 1858,  800,  598,  279,  135, 4359, 2687, 1361, 2195,
            ...
            3444, 4426, 5334,  466, 3092, 3772, 5191, 5226, 5390,  860],
           dtype='int64', length=5295)

In [83]:
import string
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

for i in X_train.index:
    for j in string.punctuation:
        X_train[i] = X_train[i].replace(j,'')
        
    X_train[i] = X_train[i].replace('\n','')
    
    X_train[i] = ' '.join([stemmer.stem(k) for k in TextBlob(X_train[i]).words])
    
    
    
        
    

In [84]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
count_vectorizer_X_train = vectorizer.fit_transform(X_train)

In [85]:
count_vectorizer_X_train.shape

(5295, 7921)

In [87]:
count_vectorizer_X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [97]:

def text_processing(df):

    for i in df.index:
        for j in string.punctuation:
            df[i] = df[i].replace(j,'')

        df[i] = df[i].replace('\n','')
        df[i] = ' '.join([stemmer.stem(k) for k in TextBlob(df[i]).words])
    
    
    return df
        
    

In [98]:
X_test = text_processing(X_test)
X_test.shape

(279,)

In [89]:
count_vectorizer_X_test = vectorizer.transform(X_test)
count_vectorizer_X_test.shape

(279, 7921)

In [94]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(count_vectorizer_X_train.toarray(), y_train)
y_pred = gnb.predict(count_vectorizer_X_test.toarray())

In [95]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8853046594982079

In [99]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(count_vectorizer_X_train.toarray(), y_train)
y_pred = clf.predict(count_vectorizer_X_test.toarray())

In [100]:
accuracy_score(y_test, y_pred)

0.8637992831541219